<a href="https://colab.research.google.com/github/SilverFoxMedia-Tesis/IA_02/blob/main/Modelo_Red_Neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt

In [2]:
#Lectura de datos
train = pd.read_csv('/dataset_peru_filtrado_v2.csv')
test = pd.read_csv('/dataset_peru_filtrado_v2.csv')

In [3]:
# Convertir a flotantes sin necesidad de reemplazar comas
train['Weight'] = pd.to_numeric(train['Weight'], errors='coerce')
train['Height'] = pd.to_numeric(train['Height'], errors='coerce')

# Convertir la edad a entero
train['Age'] = pd.to_numeric(train['Age'], downcast='integer', errors='coerce')

# Asegurar que 'Gender' y 'PhysicalActivity' sean tratados como cadenas de texto
# Esto puede no ser necesario si ya están correctamente como strings
train['Gender'] = train['Gender'].astype(str)
train['PhysicalActivity'] = train['PhysicalActivity'].astype(str)
train['Diabetes'] = train['Diabetes'].astype(str)

# Opcional: Manejar valores NaN generados por conversión fallida o datos faltantes
train.dropna(subset=['Weight', 'Height', 'Age', 'Gender', 'PhysicalActivity', 'Diabetes'], inplace=True)

In [4]:
def calcular_TMB_MIFFLIN(Weight, Height, Age, Gender, PhysicalActivity):

  # Cálculo de TMB según el género
  if Gender == 'MALE':
      partial_Tmb = (10 * Weight) + (6.25 * Height) - (5 * Age) + 5
  else: # Female
      partial_Tmb = (10 * Weight) + (6.25 * Height) - (5 * Age) - 161

  # Multiplicador según el nivel de actividad
  multiplicador = {
      'No': 1.2,
      'Slight': 1.375,
      'Moderate': 1.55,
      'Strong': 1.725,
      'Very_Strong': 1.9
  }

  # Calorías necesarias según el nivel de actividad
  TMB = partial_Tmb * multiplicador[PhysicalActivity]

  return round(TMB, 2)

In [5]:
def calcular_TMB_FAO_ONU(Weight, Age, Gender):

    if Gender == 'MALE':
        if Age >= 0 and Age <= 3:
            TMB = (60.9 * Weight) - 54
        elif Age >= 4 and Age <= 10:
            TMB = (22.7 * Weight) + 495
        elif Age >= 11 and Age <= 18:
            TMB = (17.5 * Weight) + 651
        elif Age >= 19 and Age <= 30:
            TMB = (15.3 * Weight) + 679
        elif Age >= 31 and Age <= 60:
            TMB = (11.6 * Weight) + 879
        else: # Age > 60
            TMB = (13.5 * Weight) + 487
    else: # Gender == 'FEMALE'
        if Age >= 0 and Age <= 3:
            TMB = (61.0 * Weight) - 51
        elif Age >= 4 and Age <= 10:
            TMB = (22.5 * Weight) + 499
        elif Age >= 11 and Age <= 18:
            TMB = (12.2 * Weight) + 746
        elif Age >= 19 and Age <= 30:
            TMB = (14.7 * Weight) + 496
        elif Age >= 31 and Age <= 60:
            TMB = (8.7 * Weight) + 829
        else: # edad > 60
            TMB = (10.5 * Weight) + 596

    return round(TMB, 2)

In [6]:
def calcular_TMB_harris_benedict_simplificada(Weight, Height, Age, Gender, PhysicalActivity):
    if Gender == 'MALE':
        partial_TMB = 66.5 + (13.75 * Weight) + (5.003 * Height) - (6.78 * Age)
    else:
        partial_TMB = 655 + (9.56 * Weight) + (1.85 * Height) - (4.68 * Age)
  # Multiplicador según el nivel de actividad
    multiplicador = {
      'No': 1.2,
      'Slight': 1.375,
      'Moderate': 1.55,
      'Strong': 1.725,
      'Very_Strong': 1.9
    }

    # Calorías necesarias según el nivel de actividad
    TMB = partial_TMB * multiplicador[PhysicalActivity]

    return round(TMB, 2)

In [7]:
def calcular_clasificacion_IMC(Weight, Height):
    Height_meters = Height / 100  # Convertir altura a metros
    IMC = Weight / (Height_meters ** 2)

    if IMC < 18.5:
        classificationIMC = 'Insufficient_Weight'
    elif IMC <= 24.9:
        classificationIMC = 'Normal_Weight'
    elif IMC <= 29.9:
        classificationIMC = 'Overweight'
    elif IMC <= 34.9:
        classificationIMC = 'Obesity_Type_I'
    elif IMC <= 39.9:
        classificationIMC = 'Obesity_Type_II'
    else:
        classificationIMC = 'Obesity_Type_III'

    return classificationIMC


In [8]:
def calcular_TMB(Weight, Height, Age, Gender, PhysicalActivity):
  classificationIMC = calcular_clasificacion_IMC(Weight, Height)

  if classificationIMC in ['Obesity_Type_II', 'Obesity_Type_III']:
        return calcular_TMB_MIFFLIN(Weight, Height, Age, Gender, PhysicalActivity)
  else:
      if PhysicalActivity == 'No':
          return calcular_TMB_FAO_ONU(Weight, Age, Gender)
      else:
          return calcular_TMB_harris_benedict_simplificada(Weight, Height, Age, Gender, PhysicalActivity)

In [21]:
def calcular_meta_mensual(pesoActual):
  return round(pesoActual * 0.02, 2)

In [22]:
def calcular_deficit(TMB):
  deficit = TMB - (TMB * 0.20)
  return round(deficit, 2)

In [23]:
def calcular_macronutrientes(deficit, Diabetes):
    c_carbohidratos = deficit * 0.50
    c_grasas = deficit * 0.30

    # Para diabetes, ajustamos la cantidad de proteína según el total calórico
    if Diabetes == 'YES':
        # Ajustamos directamente los gramos de proteínas basándonos en el total calórico
        g_proteinas = deficit * 0.2 / 4 / 0.8
    else:
        c_proteinas = deficit * 0.20
        g_proteinas = c_proteinas / 4

    g_carbohidratos = c_carbohidratos / 4
    g_grasas = c_grasas / 9

    return {
        "Carbohidratos_G": round(g_carbohidratos, 2),
        "Proteinas_G": round(g_proteinas, 2),
        "Grasas_G": round(g_grasas, 2)
    }

In [24]:
train['TMB'] = train.apply(lambda row: calcular_TMB(
    row['Weight'],
    row['Height'],
    row['Age'],
    row['Gender'],
    row['PhysicalActivity']
    ), axis=1)

# Asumiendo que 'train' es tu DataFrame
train['ClassificationIMC'] = train.apply(lambda row: calcular_clasificacion_IMC(row['Weight'], row['Height']), axis=1)

# Calcular el déficit para cada fila y crear la columna 'Deficit'
train['Deficit'] = train['TMB'].apply(calcular_deficit)

def get_macronutrientes(row):
    deficit = row['Deficit']
    diabetes_status = row['Diabetes']  # Asume que 'Diabetes' es el nombre de tu columna
    return calcular_macronutrientes(deficit, diabetes_status)

# Asumiendo que 'train' es tu DataFrame y ya tiene las columnas 'Deficit' y 'Diabetes'
macronutrientes = train.apply(get_macronutrientes, axis=1)

# Convertir el diccionario de resultados en un DataFrame y unirlo al DataFrame original
macronutrientes_df = pd.DataFrame(macronutrientes.tolist(), index=train.index)
# Verifica si las columnas ya existen y, de ser así, elimínalas
for col in ['Carbohidratos_G', 'Proteinas_G', 'Grasas_G']:
    if col in train.columns:
        train.drop(col, axis=1, inplace=True)

# Ahora puedes unir sin problemas
train = train.join(macronutrientes_df)

# Asumiendo que tienes un DataFrame llamado 'train' y una columna 'Weight' con el peso actual de la persona en kilogramos
train['PesoAperder'] = train['Weight'].apply(calcular_meta_mensual)


In [27]:
# Especifica un nombre para el archivo de salida
output_filename = 'train_last_modified.csv'

# Exportar el DataFrame modificado a un archivo CSV
train.to_csv(output_filename, index=False)

# Descargar el archivo a tu máquina local
files.download(output_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
train.head(60)

,Gender,Age,Diabetes,Weight,Height,AbdominalCircumference,PhysicalActivity,TMB,ClassificationIMC,Deficit,PesoAperder,Carbohidratos_G,Proteinas_G,Grasas_G
0,MALE,39,NO,75.0,171.0,95.6,No,1749.00,Overweight,1399.20,1.50,174.90,69.96,46.64
1,FEMALE,22,NO,65.9,158.1,89.5,Slight,2027.48,Overweight,1621.98,1.32,202.75,81.10,54.07
2,FEMALE,20,NO,58.3,161.5,82.9,Slight,1949.09,Normal_Weight,1559.27,1.17,194.91,77.96,51.98
3,MALE,33,NO,73.8,170.7,93.7,No,1735.08,Overweight,1388.06,1.48,173.51,69.40,46.27
4,FEMALE,38,NO,53.9,151.5,74.9,No,1297.93,Normal_Weight,1038.34,1.08,129.79,51.92,34.61
5,MALE,35,NO,90.3,176.1,103.2,No,1926.48,Overweight,1541.18,1.81,192.65,77.06,51.37
6,FEMALE,26,NO,62.4,156.1,84.5,Moderate,2198.91,Overweight,1759.13,1.25,219.89,87.96,58.64
7,FEMALE,24,NO,68.3,155.7,89.2,Moderate,2299.69,Overweight,1839.75,1.37,229.97,91.99,61.32
8,MALE,34,NO,81.5,169.0,98.0,No,1824.40,Overweight,1459.52,1.63,182.44,72.98,48.65
9,MALE,34,NO,52.9,164.6,71.5,Slight,1906.92,Normal_Weight,1525.54,1.06,190.69,76.28,50.85


carbohidratos = 187.80
proteinas = 75.12
grasas = 50.08

# Desayuno (25%)
desayuno_carbs = carbohidratos * 0.25

desayuno_prot = proteinas * 0.25

desayuno_grasas = grasas * 0.25

# Almuerzo (35%)
almuerzo_carbs = carbohidratos * 0.35

almuerzo_prot = proteinas * 0.35

almuerzo_grasas = grasas * 0.35

# Cena (40%)
cena_carbs = carbohidratos * 0.40

cena_prot = proteinas * 0.40

cena_grasas = grasas * 0.40

# Resultados
(desayuno_carbs, desayuno_prot, desayuno_grasas), (almuerzo_carbs, almuerzo_prot, almuerzo_grasas), (cena_carbs, cena_prot, cena_grasas)

In [39]:
# Cargar el nuevo dataset para revisar su contenido
food = pd.read_csv('/organized_nutrients.csv', encoding='ISO-8859-1')

# Mostrar las primeras filas del nuevo dataset para entender su estructura
food



,Calories,Carbs,category_name,Fat,Fiber,Grams,Measure,name_food,Sat.Fat
0,127,13.0,'Dairy_Products',5.0,0.0,246.0,'1 cup','Suero de la leche',4.0
1,165,11.0,'Dairy_Products',10.0,0.0,244.0,'1 cup','Leche de cabra',8.0
2,128,13.0,'Dairy_Products',4.0,1.0,250.0,'1 cup','Leche descremada',3.0
3,240,6.0,'Dairy_Products',11.0,0.0,225.0,'1 cup','Queso',10.0
4,195,6.0,'Dairy_Products',0.0,0.0,225.0,'1 cup','sin crema',0.0
...,...,...,...,...,...,...,...,...,...
149,284,8.0,'Seeds_and_Nuts',24.0,0.9,50.0,'1/3 cup','Mantequilla de maní natural',10.0
150,343,7.0,'Seeds_and_Nuts',35.0,1.1,52.0,'1/2 cup','nueces pecanas',25.0
151,280,10.0,'Seeds_and_Nuts',24.0,3.1,50.0,'1/2 cup','semillas de sésamo',13.0
152,280,10.0,'Seeds_and_Nuts',26.0,1.9,50.0,'1/2 cup','Semillas de girasol',7.0


In [58]:
# Load the dataset to view its contents
file_path = '/train_last_modified.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset and its structure
data.info(), data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29397 entries, 0 to 29396
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Gender                  29397 non-null  object 
 1   Age                     29397 non-null  int64  
 2   Diabetes                29397 non-null  object 
 3   Weight                  29397 non-null  float64
 4   Height                  29397 non-null  float64
 5   AbdominalCircumference  29397 non-null  float64
 6   PhysicalActivity        29397 non-null  object 
 7   TMB                     29397 non-null  float64
 8   ClassificationIMC       29397 non-null  object 
 9   Deficit                 29397 non-null  float64
 10  PesoAperder             29397 non-null  float64
 11  Carbohidratos_G         29397 non-null  float64
 12  Proteinas_G             29397 non-null  float64
 13  Grasas_G                29397 non-null  float64
dtypes: float64(9), int64(1), object(4)
mem

(None,
    Gender  Age Diabetes  Weight  Height  AbdominalCircumference  \
 0    MALE   39       NO    75.0   171.0                    95.6   
 1  FEMALE   22       NO    65.9   158.1                    89.5   
 2  FEMALE   20       NO    58.3   161.5                    82.9   
 3    MALE   33       NO    73.8   170.7                    93.7   
 4  FEMALE   38       NO    53.9   151.5                    74.9   
 
   PhysicalActivity      TMB ClassificationIMC  Deficit  PesoAperder  \
 0               No  1749.00        Overweight  1399.20         1.50   
 1           Slight  2027.48        Overweight  1621.98         1.32   
 2           Slight  1949.09     Normal_Weight  1559.27         1.17   
 3               No  1735.08        Overweight  1388.06         1.48   
 4               No  1297.93     Normal_Weight  1038.34         1.08   
 
    Carbohidratos_G  Proteinas_G  Grasas_G  
 0           174.90        69.96     46.64  
 1           202.75        81.10     54.07  
 2           1

In [ ]:
# Attempt to load the dataset using a different encoding (ISO-8859-1)
data_new = pd.read_csv('/organized_nutrients.csv', encoding='ISO-8859-1')

# Display the first few rows of the dataset and its structure again
data_new.info(), data_new.head()


In [60]:
# Check the unique categories available in the second dataset
unique_categories = data_new['category_name'].unique()

# Preview the structure and first few rows of both datasets again to ensure we have all necessary fields
data.head(), unique_categories


(   Gender  Age Diabetes  Weight  Height  AbdominalCircumference  \
 0    MALE   39       NO    75.0   171.0                    95.6   
 1  FEMALE   22       NO    65.9   158.1                    89.5   
 2  FEMALE   20       NO    58.3   161.5                    82.9   
 3    MALE   33       NO    73.8   170.7                    93.7   
 4  FEMALE   38       NO    53.9   151.5                    74.9   
 
   PhysicalActivity      TMB ClassificationIMC  Deficit  PesoAperder  \
 0               No  1749.00        Overweight  1399.20         1.50   
 1           Slight  2027.48        Overweight  1621.98         1.32   
 2           Slight  1949.09     Normal_Weight  1559.27         1.17   
 3               No  1735.08        Overweight  1388.06         1.48   
 4               No  1297.93     Normal_Weight  1038.34         1.08   
 
    Carbohidratos_G  Proteinas_G  Grasas_G  
 0           174.90        69.96     46.64  
 1           202.75        81.10     54.07  
 2           194.91  

In [61]:
# Calcular las metas nutricionales por comida (sin proteínas, solo carbohidratos y grasas)
data['desayuno_carbs'] = data['Carbohidratos_G'] * 0.25
data['desayuno_grasas'] = data['Grasas_G'] * 0.25

data['almuerzo_carbs'] = data['Carbohidratos_G'] * 0.35
data['almuerzo_grasas'] = data['Grasas_G'] * 0.35

data['cena_carbs'] = data['Carbohidratos_G'] * 0.40
data['cena_grasas'] = data['Grasas_G'] * 0.40

# Mostrar las primeras filas del dataset con las nuevas columnas añadidas
data.head()


,Gender,Age,Diabetes,Weight,Height,AbdominalCircumference,PhysicalActivity,TMB,ClassificationIMC,Deficit,PesoAperder,Carbohidratos_G,Proteinas_G,Grasas_G,desayuno_carbs,desayuno_grasas,almuerzo_carbs,almuerzo_grasas,cena_carbs,cena_grasas
0,MALE,39,NO,75.0,171.0,95.6,No,1749.00,Overweight,1399.20,1.50,174.90,69.96,46.64,43.7250,11.6600,61.2150,16.3240,69.960,18.656
1,FEMALE,22,NO,65.9,158.1,89.5,Slight,2027.48,Overweight,1621.98,1.32,202.75,81.10,54.07,50.6875,13.5175,70.9625,18.9245,81.100,21.628
2,FEMALE,20,NO,58.3,161.5,82.9,Slight,1949.09,Normal_Weight,1559.27,1.17,194.91,77.96,51.98,48.7275,12.9950,68.2185,18.1930,77.964,20.792
3,MALE,33,NO,73.8,170.7,93.7,No,1735.08,Overweight,1388.06,1.48,173.51,69.40,46.27,43.3775,11.5675,60.7285,16.1945,69.404,18.508
4,FEMALE,38,NO,53.9,151.5,74.9,No,1297.93,Normal_Weight,1038.34,1.08,129.79,51.92,34.61,32.4475,8.6525,45.4265,12.1135,51.916,13.844


In [62]:

# Corregir la función para ajustar al contexto actual y manejar correctamente las categorías
def select_strictly_corrected_v2(opciones, goals, categories, meal_type):
    selected_ids = []
    remaining_goals = goals.copy()
    category_pool = categories[:]

    # Procesar las categorías una por una, respetando las restricciones de exclusión
    for category in category_pool:
        filtered_options = opciones[(opciones['category_name'].str.strip("'") == category) &
                                    (opciones['Carbs'] <= remaining_goals['Carbs']) &
                                    (opciones['Fat'] <= remaining_goals['Fats'])]
        if not filtered_options.empty:
            filtered_options['diff'] = filtered_options.apply(
                lambda row: abs(row['Carbs'] - remaining_goals['Carbs']) + abs(row['Fat'] - remaining_goals['Fats']), axis=1)
            filtered_options.sort_values('diff', inplace=True)

            for _, row in filtered_options.iterrows():
                # Asegurar que no se excedan los límites de macronutrientes
                if row['Carbs'] <= remaining_goals['Carbs'] and row['Fat'] <= remaining_goals['Fats']:
                    selected_ids.append(row.name)
                    remaining_goals['Carbs'] -= row['Carbs']
                    remaining_goals['Fats'] -= row['Fat']
                    break  # Seleccionar solo un ítem por categoría y salir del bucle interno

    return selected_ids

# Categorías para cada tipo de comida
desayuno_categorias = ['Dairy_Products', 'Breads_cereals_fastfood_grains', 'Fruits_A_F']
almuerzo_categorias = ['Meat_Poultry', 'Fish_Seafood', 'Vegetables_A_E', 'Breads_cereals_fastfood_grains']
cena_categorias = ['Fish_Seafood', 'Vegetables_F_P', 'Fruits R-Z']

# Ejemplo de aplicación en las primeras 5 filas
results = []
for index, row in data.head().iterrows():
    nutrient_goals = {
        'Carbs': row['desayuno_carbs'],
        'Fats': row['desayuno_grasas']
    }
    desayuno_ids = select_strictly_corrected_v2(data_new, nutrient_goals, desayuno_categorias, 'desayuno')
    results.append(desayuno_ids)

results


<ipython-input-62-a1ec9814a295>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_options['diff'] = filtered_options.apply(
<ipython-input-62-a1ec9814a295>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_options.sort_values('diff', inplace=True)
<ipython-input-62-a1ec9814a295>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

[[1, 136, 103], [1, 134, 103], [1, 136, 103], [1, 136, 103], [0, 125, 103]]

In [63]:
# Aplicar la función de selección a todo el dataset para desayuno, almuerzo y cena
def apply_meal_selection(dataset, nutrient_data, meal_categories, meal_type):
    meal_ids = []
    for index, row in dataset.iterrows():
        nutrient_goals = {
            'Carbs': row[f'{meal_type}_carbs'],
            'Fats': row[f'{meal_type}_grasas']
        }
        meal_ids.append(select_strictly_corrected_v2(nutrient_data, nutrient_goals, meal_categories, meal_type))
    return meal_ids

# Aplicar la selección para Desayuno, Almuerzo y Cena
data['Desayuno'] = apply_meal_selection(data, data_new, desayuno_categorias, 'desayuno')
data['Almuerzo'] = apply_meal_selection(data, data_new, almuerzo_categorias, 'almuerzo')
data['Cena'] = apply_meal_selection(data, data_new, cena_categorias, 'cena')

# Mostrar las primeras filas del dataset actualizado para verificar los resultados
data[['Desayuno', 'Almuerzo', 'Cena']].head()


Se han truncado las últimas 5000 líneas del flujo de salida.
  filtered_options['diff'] = filtered_options.apply(
<ipython-input-62-a1ec9814a295>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_options.sort_values('diff', inplace=True)
<ipython-input-62-a1ec9814a295>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_options['diff'] = filtered_options.apply(
<ipython-input-62-a1ec9814a295>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

,Desayuno,Almuerzo,Cena
0,"[1, 136, 103]","[35, 59, 68, 135]","[55, 82, 122]"
1,"[1, 134, 103]","[35, 62, 68, 135]","[55, 82, 123]"
2,"[1, 136, 103]","[35, 62, 68, 135]","[55, 82, 123]"
3,"[1, 136, 103]","[35, 59, 68, 135]","[55, 82, 122]"
4,"[0, 125, 103]","[35, 52, 68]","[55, 82]"


In [64]:
# Especifica un nombre para el archivo de salida
output_filename = 'train_last_modified_v2.csv'

# Exportar el DataFrame modificado a un archivo CSV
data.to_csv(output_filename, index=False)

# Descargar el archivo a tu máquina local
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
lastData = pd.read_csv('/train_last_modified_v2.csv', encoding='ISO-8859-1')
lastData

,Gender,Age,Diabetes,Weight,Height,AbdominalCircumference,PhysicalActivity,TMB,ClassificationIMC,Deficit,...,Grasas_G,desayuno_carbs,desayuno_grasas,almuerzo_carbs,almuerzo_grasas,cena_carbs,cena_grasas,Desayuno,Almuerzo,Cena
0,MALE,39,NO,75.0,171.0,95.6,No,1749.00,Overweight,1399.20,...,46.64,43.7250,11.6600,61.2150,16.3240,69.960,18.656,"[1, 136, 103]","[35, 59, 68, 135]","[55, 82, 122]"
1,FEMALE,22,NO,65.9,158.1,89.5,Slight,2027.48,Overweight,1621.98,...,54.07,50.6875,13.5175,70.9625,18.9245,81.100,21.628,"[1, 134, 103]","[35, 62, 68, 135]","[55, 82, 123]"
2,FEMALE,20,NO,58.3,161.5,82.9,Slight,1949.09,Normal_Weight,1559.27,...,51.98,48.7275,12.9950,68.2185,18.1930,77.964,20.792,"[1, 136, 103]","[35, 62, 68, 135]","[55, 82, 123]"
3,MALE,33,NO,73.8,170.7,93.7,No,1735.08,Overweight,1388.06,...,46.27,43.3775,11.5675,60.7285,16.1945,69.404,18.508,"[1, 136, 103]","[35, 59, 68, 135]","[55, 82, 122]"
4,FEMALE,38,NO,53.9,151.5,74.9,No,1297.93,Normal_Weight,1038.34,...,34.61,32.4475,8.6525,45.4265,12.1135,51.916,13.844,"[0, 125, 103]","[35, 52, 68]","[55, 82]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29392,FEMALE,35,NO,66.4,149.6,96.3,Slight,1928.77,Overweight,1543.02,...,51.43,48.2200,12.8575,67.5080,18.0005,77.152,20.572,"[1, 136, 103]","[35, 62, 68, 135]","[55, 82, 123]"
29393,FEMALE,21,NO,60.4,156.4,84.0,No,1383.88,Normal_Weight,1107.10,...,36.90,34.5975,9.2250,48.4365,12.9150,55.356,14.760,"[0, 124, 103]","[35, 52, 68]","[55, 82]"
29394,MALE,36,NO,63.2,161.9,88.7,Moderate,2327.18,Normal_Weight,1861.74,...,62.06,58.1800,15.5150,81.4520,21.7210,93.088,24.824,"[1, 134, 108]","[35, 55, 68, 136]","[55, 82, 123]"
29395,MALE,34,NO,78.7,167.0,89.9,Slight,2411.21,Overweight,1928.97,...,64.30,60.2800,16.0750,84.3920,22.5050,96.448,25.720,"[1, 134, 108]","[35, 55, 68, 136]","[55, 82, 123]"


In [68]:
# Cargar el nuevo dataset para revisar su contenido
wk = pd.read_csv('/Workout.csv', encoding='ISO-8859-1')

# Mostrar las primeras filas del nuevo dataset para entender su estructura
wk

,BodyPart,RepsPerSet,Sets,TypeExercise,Workout
0,Chest,8-12,3-4,Upper_Chest,Press inclinado con mancuernas
1,Chest,10-15,3-4,Upper_Chest,Cruces de cables inclinados
2,Chest,10-15,3-4,Upper_Chest,Volantes inclinados con mancuernas
3,Chest,8-12,3-4,Lower_Chest,Press declinado con mancuernas
4,Chest,10-15,3-4,Lower_Chest,Rechazar cruces de cables
5,Chest,10-15,3-4,Lower_Chest,Declinar aperturas con mancuernas
6,Chest,10-15,3-4,Chest_Isolation,Moscas en el pecho
7,Chest,10-15,3-4,Chest_Isolation,Cruces de cables
8,Chest,10-15,3-4,Chest_Isolation,Aperturas con mancuernas
9,Back,8-12,3-4,Upper,Dominadas


In [69]:
# Actualizar para seleccionar aleatoriamente entre 4 y 5 ejercicios por cada fila en el DataFrame

# Generar números aleatorios para la cantidad de ejercicios por rutina (4 a 5)
num_ejercicios = np.random.randint(4, 6, size=len(lastData))

# Actualizar la columna 'Rutina' con los nuevos ejercicios seleccionados
lastData['Rutina'] = [list(np.random.choice(wk.index, size=n, replace=False)) for n in num_ejercicios]

# Mostrar las primeras filas del DataFrame modificado para verificar la actualización
lastData.head()


,Gender,Age,Diabetes,Weight,Height,AbdominalCircumference,PhysicalActivity,TMB,ClassificationIMC,Deficit,...,desayuno_carbs,desayuno_grasas,almuerzo_carbs,almuerzo_grasas,cena_carbs,cena_grasas,Desayuno,Almuerzo,Cena,Rutina
0,MALE,39,NO,75.0,171.0,95.6,No,1749.00,Overweight,1399.20,...,43.7250,11.6600,61.2150,16.3240,69.960,18.656,"[1, 136, 103]","[35, 59, 68, 135]","[55, 82, 122]","[37, 5, 40, 30, 33]"
1,FEMALE,22,NO,65.9,158.1,89.5,Slight,2027.48,Overweight,1621.98,...,50.6875,13.5175,70.9625,18.9245,81.100,21.628,"[1, 134, 103]","[35, 62, 68, 135]","[55, 82, 123]","[31, 19, 29, 10, 12]"
2,FEMALE,20,NO,58.3,161.5,82.9,Slight,1949.09,Normal_Weight,1559.27,...,48.7275,12.9950,68.2185,18.1930,77.964,20.792,"[1, 136, 103]","[35, 62, 68, 135]","[55, 82, 123]","[5, 37, 15, 38, 0]"
3,MALE,33,NO,73.8,170.7,93.7,No,1735.08,Overweight,1388.06,...,43.3775,11.5675,60.7285,16.1945,69.404,18.508,"[1, 136, 103]","[35, 59, 68, 135]","[55, 82, 122]","[5, 37, 45, 31, 12]"
4,FEMALE,38,NO,53.9,151.5,74.9,No,1297.93,Normal_Weight,1038.34,...,32.4475,8.6525,45.4265,12.1135,51.916,13.844,"[0, 125, 103]","[35, 52, 68]","[55, 82]","[14, 50, 4, 17, 7]"


In [70]:
# Especifica un nombre para el archivo de salida
output_filename = 'train_last_modified_v3.csv'

# Exportar el DataFrame modificado a un archivo CSV
lastData.to_csv(output_filename, index=False)

# Descargar el archivo a tu máquina local
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>